In [1]:
using DataFrames
using CSV

In [2]:
function limpiarTexto(f)

    lines = readlines(f)
    fecha_in = findfirst(t -> occursin("Fecha", t), lines)
    lines[end] = "" # eliminar ultima linea
    # si existe esa esctructura eliminarla
    if lines[end - 1] == ";;;;;;;;;;;;;;;;"
        lines[end - 1] = "" 
    end

    lines = lines[fecha_in: end] # obtener solo las lineas a partir de la fecha
    lines[1] = replace(lines[1], "\xe1" => "") # sustituir caracteres especiales  
    string = join(lines, "\n") # vector a string para leerlo como csv
    string = replace(string, ";;" => ";") # eliminar duplicados
    
    return string
end

limpiarTexto (generic function with 1 method)

In [3]:
arr = []
carpeta = "C:/Users/Matias/Desktop/buscar dato"
for i in 1:length(readdir(carpeta))
    archivo = carpeta * "/" * string(i) * "ReporteRegistros.csv"
    push!(arr, archivo)
end

In [4]:
vector = []
for j in 1:length(readdir(carpeta))
    archivo = carpeta * "/" * string(j) * "ReporteRegistros.csv"
    open(archivo, "r") do f

        string = limpiarTexto(f)
        global df = CSV.read(IOBuffer(string), DataFrame; delim = ";", header = true) # string a csv
        select!(df, "Fecha Hora", "D.Mx kW") 
        
        posicion = findfirst(t -> startswith("31/05/22 20:30:00", t), df[!, 1])
        if posicion != nothing
            push!(vector, df[posicion, 2])
        else
            push!(vector, "sin dato")
        end
        
    end
end

In [5]:
open("C:/Users/Matias/Desktop/out.txt", "a") do file
    for k in 1:length(vector)
        write(file, replace(vector[k], "," => ".") * "\n")
    end
end

---------------------------------------------------------------------------------------------------------------------------------------------------